# Accessing a Generative AI Model through OpenAI API

Version: 2025-10-5

Generative AI models are machine learning models that are capable of generating content.
This notebook is going to focus on the most commonly-used type of generative AI models&mdash;the 
ones that generate text. These models are called *large language models*, or 'LLM' for short.
We will not be going into the technical details of how a LLM works at this moment, but rather focus
on how to use an LLM as an end user.

### A. Accessing an LLM through Python
Although it is very convenient to access an LLM through a browser interface for one-off task, 
for repetitive task you will generating want to access the model using a program. Model providers 
will generally provide such access through an *application programming interface*, or 'API' for short.
An API defines what sorts of interactions are possible with the model, and how input and output data 
should look like. 

The API used by OpenAI, the provider of ChatGPT, is the de facto standard supported by most models.

The first step is to set up an OpenAI client. The client requires two piece of information:
1. `base_url`: this is the web address of the model provider. If you use OpenAI's model, this can be omited.
2. `api_key`: this is a string of text unique to your user account with the model provider. 

In [ ]:
from openai import OpenAI



In a production environment, it is best to save this piece of information as an environment variable, so that it will not be saved in your notebook. The easiest way to do so when working in a Jupyter notebook is to put the key in a filed named `.env`, then use the `dotenv` library to set the environment variable.

In [ ]:
from openai import OpenAI
from dotenv import load_dotenv
import os

# Load the .env file containing environment variables.
# Default is to look for it in the current working directory.


# The API key should be saved as an environment variable OPENAI_API_KEY
# Optionally set the base_url as OPENAI_BASE_URL in the same file


### B. Model List

To see what models the provider support:

### C. Calling the Model

To call a model, use 
```python
response = client.chat.completions.create(...)
```
You will need to provide a few pieces of information:
- `model`: the name of the model.
- `messages`: a list containing the conversation history. Each message should be dictionary with a `role` and a `content`. `role` can be one of:
    - `system`: system prompt. Use this to give the model general guidelines that it should follow strictly.
    - `user`: prompt entered by the user. 
    - `assistant`: response from the model. 
    - `tool`: response from tools the model can use.
- Additional settings such as temperature and number of samples:
    - `temperature`: Adjust the randomness of the response. 0 is the lowest setting.
      Default is model specfic, around 0.6-0.8.
    - `n`: number of responses you want the model to generate. Default is 1.

The response will be recorded in
```python
response.choices[0].message.content
```

In [ ]:
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

client = OpenAI()



### D. Example: Scrapping Webpage and Extract Information with LLM

We first scrape the webpage with Selenium + Beautiful Soup,
but unlike before, we do not manually find the elements we need.
Instead, we export a stripped version of the webpage's source code,
then uses an LLM to extract the information we need. 

Because LLMs can only process a limited amount of text input, called 
*context length*, and slows down as the context length goes up, 
it is usually wise to use Beautiful Soup to locate
as precious as possible what you need, before passing the source code
to the LLM.

Scrape the webpage in full:

In [20]:
# Only works with static content.
# See A6-Data-Scrping-Completed for a version that works with dynamic content.
import requests
def get_page_source(url, strip=False):
    # Function to access a page and save all horses into a list
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    return soup.get_text(strip=strip)   

Now we feed the source code to the LLM:

In [ ]:
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()



You can strip white space by setting `strip = True` in `get_page_source()`, 
but it appears to drastically lower the quality of the LLM's output:

In [ ]:
from openai import OpenAI
import time
from dotenv import load_dotenv

page_source = get_page_source('http://racing.hkjc.com/racing/Info/Meeting/Results/english/Local/20151213/ST',
                              strip=True)

load_dotenv()
client = OpenAI()

start = time.time()
response = client.chat.completions.create(
  model="default",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": f"""The following webpage contains horse racing results. 
                                Please extract the result table and return it in comma-delimited
                                CSV format. No explanation is needed.
                                {page_source}
                                """},
  ],
)
output = response.choices[0].message.content
print("Processing took: {:.2f}".format(time.time() - start))
print(output)

To speed up the processing and make the model's output more accurate, you can use Beauitful Soup to locate the table, and provide only the source code of the table to the LLM:

Let us try again:

In [26]:
page_source = get_page_source('http://racing.hkjc.com/racing/Info/Meeting/Results/english/Local/20151213/ST',
                              strip=True)
start = time.time()
response = client.chat.completions.create(
  model="default",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": f"""The following webpage contains horse racing results. 
                                Please extract the result table and return it in comma-delimited
                                CSV format. No explanation is needed.
                                {page_source}
                                """},
  ],
)
output = response.choices[0].message.content
print("Processing took: {:.2f}".format(time.time() - start))
print(output)

Processing took: 12.99
Pla.,Horse No.,Horse,Jockey,Trainer,Act. Wt.,Declar. Horse Wt.,Dr.,LBW,Running Position,Finish Time,Win Odds
1,10,JOLLY JOLLY,K Teetan,P O'Sullivan,114,1214,13,-,"1 1 1 1",1:22.05,2.6
2,8,PEOPLE'S KNIGHT,T Berry,J Moore,119,1163,8,2,"2 4 2 2",1:22.39,5.7
3,12,RUN FORREST,J Moreira,C S Shum,115,1135,10,3,"14 10 10 3",1:22.54,3.9
4,4,MODERN TSAR,B Prebble,W Y So,123,1101,11,4-3/4,"9 13 13 4",1:22.80,13
5,3,MAGNETISM,G Lerena,D E Ferraris,125,1130,3,4-3/4,"7 6 6 5",1:22.81,52
6,1,ENORMOUS HONOUR,N Rawiller,Y S Tsui,131,1127,9,5-1/4,"11 12 12 6",1:22.87,10
7,9,HAPPY JOURNEY,H W Lai,S Woods,114,1040,5,6-1/2,"5 7 7 7",1:23.08,121
8,14,WINGOLD,M L Yeung,A Lee,111,1154,12,6-1/2,"13 14 14 8",1:23.11,331
9,11,OVETT,H N Wong,A T Millard,105,1153,4,7-1/4,"3 2 3 9",1:23.20,41
10,5,PAKISTAN BABY,D Whyte,A S Cruz,121,1023,7,7-1/4,"6 11 11 10",1:23.22,12
11,6,SUPER FLUKE,M Demuro,D Cruz,120,1109,14,7-3/4,"4 3 4 11",1:23.27,65
12,7,JUN GONG,C Y Ho,C H Yip,115,1147,6,10-1/4,"10 9 

Finally, we write the output to file:

In [ ]:
with open("result.csv", "a") as f:
  f.write(output)

### E. Outputing to File

Because the LLM has done the formatting, we directly write
the output to file:

### F. Scrape Multiple Pages

We can put everything together in one function
and use it in a loop to save the information we need:

In [ ]:
# Function to scrape results

def scrape_webpage(url, prompt):

        
        

In [ ]:
# The first part of the URL of data source
url_front = "http://racing.hkjc.com/racing/Info/Meeting/Results/english/Local/"
prompt = """The following webpage may contain horse racing results in a table. 
            If it does, please extract the result table and return it in 
            comma-delimited CSV format. If not, just return an empty string.
            No explanation is needed.
            """

# Copy the loop from above and incorporate the csv-saving code
for year in range(2017,2018):
    for month in range(1,2):
        for day in range(1,15):
            
            # Convert month and day to 2-digit representation
            month_2d = '{:02d}'.format(month)
            day_2d = '{:02d}'.format(day)
            
            # Full URL of data source
            url = url_front + str(year) + month_2d + day_2d
            
            # Print the URL so we know the progress so far
            print("Trying:",url)
            
            # Call our function to fetch and process data given the URL
            output = scrape_webpage(url, prompt)
            
            # Only save if there is something in content
            if output is not None:
                filepath = str(year)+month_2d+day_2d+".csv"
                
                # Save to file
                with open(filepath, "w") as f:
                    f.write(output)
                    print(f"{filepath} saved.")

